In [20]:
import normals_lib
import pointareas as pareas
import kaolin as kal
import torch
import numpy as np
import meshplot as mp
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib tk
#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
mesh = kal.rep.TriangleMesh.from_obj("stat_triangular.obj")
#mesh = kal.rep.TriangleMesh.from_obj("torus.obj")
#mesh = kal.rep.TriangleMesh.from_obj("simplified_bunny.obj")

In [41]:
view_coords = torch.load("view_coordinates.pt").to(device=torch.device('cuda'))
verts = mesh.vertices.clone()
xmed = torch.median(verts[:,0])
ymed = torch.median(verts[:,1])
zmed = torch.median(verts[:,2])
verts[:,0] -= xmed
verts[:,1] -= ymed
verts[:,2] -= zmed
verts /= torch.max(torch.norm(verts, dim=1))
#mesh.vertices = verts

In [2]:
normals = normals_lib.compute_simple_vertex_normals(mesh).cpu()
pointareas, cornerareas = pareas.compute_pointareas(mesh)
normals_lib.visualize_normals(mesh, normals)
normals = normals.to(device=torch.device('cuda'))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-20.83300…

In [3]:
# Calculate principal curvatures and curvature directions
import curvature
import importlib

k1, k2, e1, e2 = curvature.compute_curvatures(mesh, normals=normals, pointareas=pointareas,
                                              cornerareas=cornerareas)

In [38]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, curvs=(k1,k2,e1,e2), normals=normals, pointareas=pointareas,
                          cornerareas=cornerareas)

In [18]:
importlib.reload(curvature)

<module 'curvature' from '/home/danniccs/rtsc_to_torch/curvature.py'>

In [42]:
DwKr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
kr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
for i in range(0, 12):
    DwKr[i], kr[i] = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc,
                                        view_coords=view_coords[i])
    kupper = int(DwKr[i].shape[0] * 0.9)
    klower = int(DwKr[i].shape[0] * 0.1)
    dwupper, _ = torch.kthvalue(DwKr[i], kupper)
    dwlower, _ = torch.kthvalue(DwKr[i], klower)
    DwKr[i][DwKr[i] > dwupper] = dwupper
    DwKr[i][DwKr[i] < dwlower] = dwlower
    kupper = int(kr[i].shape[0] * 0.9)
    klower = int(kr[i].shape[0] * 0.1)
    krupper, _ = torch.kthvalue(kr[i], kupper)
    krlower, _ = torch.kthvalue(kr[i], klower)
    #kr[i][kr[i] > krupper] = krupper
    #kr[i][kr[i] < krlower] = krlower

In [22]:
DwKr_alt = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
kr_alt = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
for i in range(0, 12):
    DwKr_alt[i], kr_alt[i] = dcurv.compute_DwKr_alt(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc,
                                                    view_coords=view_coords[i])
    kupper = int(DwKr_alt[i].shape[0] * 0.9)
    klower = int(DwKr_alt[i].shape[0] * 0.1)
    dwupper, _ = torch.kthvalue(DwKr_alt[i], kupper)
    dwlower, _ = torch.kthvalue(DwKr_alt[i], klower)
    DwKr_alt[i][DwKr_alt[i] > dwupper] = dwupper
    DwKr_alt[i][DwKr_alt[i] < dwlower] = dwlower
    kupper = int(kr_alt[i].shape[0] * 0.9)
    klower = int(kr_alt[i].shape[0] * 0.1)
    krupper, _ = torch.kthvalue(kr_alt[i], kupper)
    krlower, _ = torch.kthvalue(kr_alt[i], klower)
    #kr_alt[i][kr_alt[i] > krupper] = krupper
    #kr_alt[i][kr_alt[i] < krlower] = krlower

In [44]:
tridc = torch.zeros(dc.shape)
dwtri = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
krtri = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
with open("/home/danniccs/rtsc-1.6/dcurv", "r") as myfile:
    for i in range(0, dc.shape[0]):
        l = myfile.readline()
        tridc[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])

for i in range(0, 12):
    dwtri[i], krtri[i] = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=tridc.cuda(),
                                            view_coords=view_coords[i])

In [45]:
for i in range(0, 1):
    kupper = int(dwtri[i].shape[0] * 0.9)
    klower = int(dwtri[i].shape[0] * 0.1)
    dwupper, _ = torch.kthvalue(dwtri[i], kupper)
    dwlower, _ = torch.kthvalue(dwtri[i], klower)
    dwtri[i][dwtri[i] > dwupper] = dwupper
    dwtri[i][dwtri[i] < dwlower] = dwlower
    #mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=dwtri[i].to(device=torch.device('cpu')).numpy(),
    #        shading={'colormap': "jet_r"})

In [4]:
trik1 = torch.zeros(k1.shape).cuda()
trik2 = torch.zeros(k2.shape).cuda()
trie1 = torch.zeros(e1.shape).cuda()
trie2 = torch.zeros(e2.shape).cuda()

with open("/home/danniccs/rtsc-1.6/k1curv", "r") as myfile:
    for i in range(0, k1.shape[0]):
        l = myfile.readline()
        trik1[i] = float(l)

with open("/home/danniccs/rtsc-1.6/k2curv", "r") as myfile:
    for i in range(0, k2.shape[0]):
        l = myfile.readline()
        trik2[i] = float(l)

with open("/home/danniccs/rtsc-1.6/pdir1", "r") as myfile:
    for i in range(0, e1.shape[0]):
        l = myfile.readline()
        trie1[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])

with open("/home/danniccs/rtsc-1.6/pdir2", "r") as myfile:
    for i in range(0, e2.shape[0]):
        l = myfile.readline()
        trie2[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])

In [26]:
import shared_algs
importlib.reload(shared_algs)
ndotv = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
krf = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest_num = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest_den = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))

tridc = tridc.cuda()
for i in range(0, 12):
    ndotv[i], krf[i], _, sctest_num[i], sctest_den[i] = shared_algs.compute_perview(mesh, normals=normals,
                                                                                    curvs=(trik1,trik2,trie1,trie2),
                                                                                    dcurv=tridc,
                                                                                    view_coords=view_coords[i])
    #sctest_den[i][sctest_den[i] == 0] = 0.01
    sctest[i] = sctest_num[i] / sctest_den[i]
    kupper = int(sctest[i].shape[0] * 0.9)
    klower = int(sctest[i].shape[0] * 0.1)
    scupper, _ = torch.kthvalue(sctest[i], kupper)
    sclower, _ = torch.kthvalue(sctest[i], klower)
    sctest[i][sctest[i] > scupper] = scupper
    sctest[i][sctest[i] < sclower] = sclower

In [10]:
mp.jupyter()
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k1.cpu().numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=trik1.cpu().numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [11]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k2.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=trik2.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [56]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=e1.to(device=torch.device('cpu')).numpy(),
               s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=trie1.to(device=torch.device('cpu')).numpy(),
               s=[1,2,1], data=d)

In [51]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k1.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=trik1.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [10]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[1].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[1].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[1].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[1].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [17]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[2].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[2].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[2].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[2].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [40]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[3].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[3].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest[3].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[3].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [94]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[4].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[4].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[4].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[4].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [87]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[5].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[5].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[5].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[5].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [88]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[6].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[6].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[6].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[6].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [89]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[7].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[7].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[7].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[7].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [90]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[8].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[8].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[8].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[8].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [91]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[9].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[9].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[9].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[9].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [92]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[10].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[10].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[10].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[10].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [93]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[11].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr_alt[11].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,1], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest_num[1].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,2], data=d)
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[11].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[2,2,3], data=d)

In [46]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de DwKr\n')

hplot = sns.histplot(ax=axes[0], data=DwKr[0].cpu().numpy())
hplot.set(xlabel='Valor de DwKr')
axes[0].set_title('Nuestra implementación')

hplot = sns.histplot(ax=axes[1], data=dwtri[0].cpu().numpy())
hplot.set(xlabel='Valor de DwKr')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [48]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de DwKr\n')

hplot = sns.histplot(ax=axes[0], data=DwKr[0].cpu().numpy())
hplot.set(xlabel='Valor de DwKr')
axes[0].set_title('Nuestra implementación')

hplot = sns.histplot(ax=axes[1], data=sctest[0].cpu().numpy())
hplot.set(xlabel='Valor de DwKr')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [47]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de curvaturas radiales\n')

hplot = sns.histplot(ax=axes[0], data=kr[0].cpu().numpy())
hplot.set(xlabel='Valor de kr')
axes[0].set_title('Nuestra implementación')

hplot = sns.histplot(ax=axes[1], data=krtri[0].cpu().numpy())
hplot.set(xlabel='Valor de kr')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [47]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de curvaturas radiales\n')

hplot = sns.histplot(ax=axes[0], data=kr[0].cpu().numpy())
hplot.set(xlabel='Valor de kr')
axes[0].set_title('Nuestra implementación')

hplot = sns.histplot(ax=axes[1], data=krf[0].cpu().numpy())
hplot.set(xlabel='Valor de kr')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [57]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de curvatura principal k1\n')

hplot = sns.histplot(ax=axes[0], data=k1.cpu().numpy())
hplot.set(xlabel='Valor de k1')
axes[0].set_title('Implementación en Python')

hplot = sns.histplot(ax=axes[1], data=trik1.cpu().numpy())
hplot.set(xlabel='Valor de k1')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [31]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de curvatura principal k2\n')

hplot = sns.histplot(ax=axes[0], data=k2.cpu().numpy())
hplot.set(xlabel='Valor de k2')
axes[0].set_title('Implementación en Python')

hplot = sns.histplot(ax=axes[1], data=trik2.cpu().numpy())
hplot.set(xlabel='Valor de k2')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [36]:
df = pd.DataFrame(e1.cpu().numpy(), columns=['c1', 'c2', 'c3'])
ax = plt.subplot(111, polar=True)
sns.histplot(data=df, ax=ax)

<PolarAxesSubplot:ylabel='Count'>